In [3]:
from xgboost import XGBRegressor
import pandas as pd
import numpy as np
from joblib import dump, load
from os import path
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from os.path import join, isfile
from os import listdir

In [4]:
time_precision = 4   # jagame päeva xh suurusteks plokkideks

X_test = pd.read_csv("../../data/X_test.csv")
X_test = X_test.apply(pd.to_numeric, errors='ignore', downcast='integer')
X_test.ToimAeg = X_test.ToimAeg.values.astype('datetime64[D]')
X_test.ToimAeg = X_test['ToimAeg'].apply(lambda x: x.replace(hour=x.hour-x.hour%time_precision, minute=0, second=0))
X_test["Aasta"] = X_test['ToimAeg'].dt.year
X_test["Kuu"] = X_test['ToimAeg'].dt.month
X_test["Päev"] = X_test['ToimAeg'].dt.day
X_test["Tund"] = X_test['ToimAeg'].dt.hour
X_test.drop(columns="ToimAeg", inplace=True)
X_test = X_test.apply(pd.to_numeric, errors='ignore', downcast='integer')
X_test.head()

y_test = pd.read_csv("../../data/y_test.csv")
y_test = y_test.apply(pd.to_numeric, errors='ignore', downcast='integer')
y_test.head()


y_test.drop(columns=["Kahjusumma", "Koht"], inplace=True)


In [10]:
# testing of thing
# loading models

onlyfiles = [f for f in listdir("../cleaner_models/trained_models/xgbr") if isfile(join("../cleaner_models/trained_models/xgbr", f))]

models_dict = dict()
for file in onlyfiles:
    file_tot_path = join("../cleaner_models/trained_models/xgbr", file)
    if not "LIIK_KONFLIKTOMAVOLI" in file:
        models_dict[file.split("_", 1)[1].split(".")[0]] = load(file_tot_path)
    else:
        models_dict["LIIK_KONFLIKT/OMAVOLI"] = load(file_tot_path)
print("loaded")
xgb_models = list()
for col in y_test.columns:
    xgb_models.append(models_dict[col])

# loading of rfc
rfc = load("../cleaner_models/trained_models/rfc.joblib")
print("loaded")

loaded
loaded


In [13]:
for col, model in list(zip(y_test.columns, xgb_models)):
    print(metrics.roc_auc_score(y_test[col], model.predict(X_test)), col, sep="\t")

sum([metrics.roc_auc_score(y_test[col], model.predict(X_test)) for col, model in list(zip(y_test.columns, xgb_models))]) / len(y_test.columns)

0.9343475860407968	LIIK_AVALIKU_KORRA_RIKKUMINE
0.9057746665833776	LIIK_JALGRATTA_MOPEEDI_VARGUS
0.8834619152483184	LIIK_KEHALINE_VAARKOHTLEMINE
0.7108186405233774	LIIK_KELMUS
0.5348208796924927	LIIK_KONFLIKT/OMAVOLI
0.8357859913301393	LIIK_KOOLIVAGIVALD
0.4411361371730744	LIIK_LIIKLUSRIKKUMINE
0.8266827541827542	LIIK_MOOTORSOIDUKI_VARGUS
0.7162307715350342	LIIK_MUU
0.8693620597934607	LIIK_PISIVARGUS
0.7998817282547305	LIIK_ROOVIMINE
0.6251335298893305	LIIK_TULEKAHJU
0.8645900844901134	LIIK_VANDALISM
0.7760734419297979	LIIK_VARGUS


0.7660071561904856

In [17]:
probs_rfc = rfc.predict(X_test)

# predict jaoks df tegemine
df_proba = pd.DataFrame(probs_rfc, columns=y_test.columns) # @rasmus ma ei oska kirjelda miks see rida vajalik aga igastahes see tuleb sellest probs_rfc kujust mis ei sobinud
for col in df_proba.columns:
    print(metrics.roc_auc_score(y_test[col], df_proba[col]), col, sep="\t")

sum([metrics.roc_auc_score(y_test[col], df_proba[col]) for col in y_test.columns]) / len(y_test.columns)


0.8025482267969134	LIIK_AVALIKU_KORRA_RIKKUMINE
0.5679268045101041	LIIK_JALGRATTA_MOPEEDI_VARGUS
0.5626638795830777	LIIK_KEHALINE_VAARKOHTLEMINE
0.5211477795653687	LIIK_KELMUS
0.5	LIIK_KONFLIKT/OMAVOLI
0.694380276066621	LIIK_KOOLIVAGIVALD
0.4999358727715788	LIIK_LIIKLUSRIKKUMINE
0.49987129987129986	LIIK_MOOTORSOIDUKI_VARGUS
0.5116298757370017	LIIK_MUU
0.7501129037747838	LIIK_PISIVARGUS
0.506784690537021	LIIK_ROOVIMINE
0.5	LIIK_TULEKAHJU
0.5460968984940255	LIIK_VANDALISM
0.691317406735374	LIIK_VARGUS


0.5824582796030836

In [22]:
def applyCutoff(df, cutoff):
    a = df.copy()
    for col in a.columns:
        a[col] = a[col].apply(lambda x: 1 if x > cutoff else 0)
    return a
best_cuttoff = {'LIIK_AVALIKU_KORRA_RIKKUMINE': 0.48000000000000015, 'LIIK_JALGRATTA_MOPEEDI_VARGUS': 0.5000000000000002, 'LIIK_KEHALINE_VAARKOHTLEMINE': 0.5600000000000003, 'LIIK_KELMUS': 0.7300000000000004, 'LIIK_KONFLIKT/OMAVOLI': 0.7600000000000005, 'LIIK_KOOLIVAGIVALD': 0.3, 'LIIK_LIIKLUSRIKKUMINE': 0.36000000000000004, 'LIIK_MOOTORSOIDUKI_VARGUS': 0.5300000000000002, 'LIIK_MUU': 0.7700000000000005, 'LIIK_PISIVARGUS': 0.5400000000000003, 'LIIK_ROOVIMINE': 0.6000000000000003, 'LIIK_TULEKAHJU': 0.5100000000000002, 'LIIK_VANDALISM': 0.7300000000000004, 'LIIK_VARGUS': 0.5300000000000002}
# xgbr accuracy
for col, model in list(zip(y_test.columns, xgb_models)):
    print(metrics.accuracy_score(y_test[col], applyCutoff(pd.DataFrame(model.predict(X_test)), best_cuttoff[col])), col, sep="\t")

sum([metrics.accuracy_score(y_test[col], applyCutoff(pd.DataFrame(model.predict(X_test)), best_cuttoff[col])) for col, model in list(zip(y_test.columns, xgb_models))]) / len(y_test.columns)

0.9490396927016646	LIIK_AVALIKU_KORRA_RIKKUMINE
0.9742637644046095	LIIK_JALGRATTA_MOPEEDI_VARGUS
0.970550576184379	LIIK_KEHALINE_VAARKOHTLEMINE
0.9939820742637644	LIIK_KELMUS
0.9975672215108835	LIIK_KONFLIKT/OMAVOLI
0.9980793854033291	LIIK_KOOLIVAGIVALD
0.9975672215108835	LIIK_LIIKLUSRIKKUMINE
0.9948783610755442	LIIK_MOOTORSOIDUKI_VARGUS
0.9608194622279129	LIIK_MUU
0.8115236875800256	LIIK_PISIVARGUS
0.9901408450704225	LIIK_ROOVIMINE
0.9988476312419975	LIIK_TULEKAHJU
0.9280409731113957	LIIK_VANDALISM
0.7015364916773368	LIIK_VARGUS


0.9476312419974392